Connecting model to google sheet

In [1]:
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np

In [2]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
account = ServiceAccountCredentials.from_json_keyfile_name("nifty50-378307-4abc522375d9.json",scope)
access = gspread.authorize(account)

Dropdown for Stock Price Predictions

In [3]:
Symbol_li = ["ADANIENT", "ADANIPORTS", "APOLLOHOSP", "ASIANPAINT", "AXISBANK", "BAJAJ-AUTO", "BAJFINANCE", "BAJAJFINSV", "BPCL", "BHARTIARTL",
        "BRITANNIA", "CIPLA", "COALINDIA", "DIVISLAB", "DRREDDY", "EICHERMOT", "GRASIM", "HCLTECH", "HDFCBANK", "HDFCLIFE",
        "HEROMOTOCO", "HINDALCO", "HINDUNILVR", "HDFC", "ICICIBANK", "ITC", "INDUSINDBK", "INFY", "JSWSTEEL", "KOTAKBANK",
        "LT", "M&M","MARUTI","NTPC","NESTLEIND","ONGC","POWERGRID","RELIANCE","SBILIFE","SBIN",
        "SUNPHARMA","TCS","TATACONSUM","TATAMOTORS","TATASTEEL","TECHM","TITAN","UPL","ULTRACEMCO","WIPRO"]

In [4]:
# stock=[]
# for i in Symbol_li:
#     stock.append({"userEnteredValue" : i})

In [5]:
# MAIN SPREADSHEET
Project = access.open_by_url('https://docs.google.com/spreadsheets/d/1HulVAUOGck6-1QqrznFTs843iIRnDIqZ9icGgBiSz5Y/edit#gid=0')

In [6]:
# sheetName = "Sheet1" # Sheet_Name.
# sheetId = Project.worksheet(sheetName).id # Extracting Sheet_ID
# body = {
#     "requests": [
#         {
#             "updateCells": {
#                 "range": {
#                     "sheetId": sheetId,
#                     "startRowIndex": 3,
#                     "endRowIndex": 4,
#                     "startColumnIndex": 1,
#                     "endColumnIndex": 2
#                 },
#                 "rows": [
#                     {
#                         "values": [
#                             {
#                                 "dataValidation": {
#                                     "condition": {
#                                         "values": stock,
#                                         "type": "ONE_OF_LIST"
#                                     },
#                                     "showCustomUi": True
#                                 }
#                             }
#                         ]
#                     }
#                 ],
#                 "fields": "dataValidation"
#             }
#         }
#     ]
#  }
# Project.batch_update(body)

In [7]:
N50 = Project.get_worksheet(0)
N50

<Worksheet 'Sheet1' id:0>

In [8]:
data = pd.DataFrame(N50.get_all_records())

Stock Prediction LSTM Model

In [9]:
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tfl
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [10]:
# def stockPredictionModel(Company_name, Number_of_share, Buying_date, Selling_date):
#     df = pd.read_csv('Nifty50.csv')
#     df.head()
#     df["Date"] = pd.to_datetime(df["Date"])
#     # name_of_stock = input("Enter Name Of Stock")
#     # number_of_share = int(input("Number Of Share"))
#     Y = df[df["Symbol"]==Company_name][["Close"]]
#     # start_date = input("Start Date")
#     # end_date = input("End Date")
#     Buying_date = pd.to_datetime(Buying_date)
#     Selling_date = pd.to_datetime(Selling_date)



#     scaler=MinMaxScaler(feature_range=(0,1))
#     Y=scaler.fit_transform(np.array(Y).reshape(-1,1))



#     training_size=int(len(Y)*0.65)
#     test_size=len(Y)-training_size
#     train_data,test_data=Y[0:training_size,:],Y[training_size:len(Y),:1]


#     # convert an array of values into a dataset matrix
#     def create_dataset(dataset, time_step):
#         dataX, dataY = [], []
#         for i in range(len(dataset)-time_step-1):
#             a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
#             dataX.append(a)
#             dataY.append(dataset[i + time_step, 0])
#         return np.array(dataX), np.array(dataY)


#     time_step = 100
#     X_train, y_train = create_dataset(train_data, time_step)
#     X_test, ytest = create_dataset(test_data, time_step)


#     X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
#     X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

#     ### Create the Stacked LSTM model
#     model=Sequential()
#     model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
#     model.add(LSTM(50,return_sequences=True))
#     model.add(LSTM(50))
#     model.add(Dense(1))
#     model.compile(loss='mean_squared_error',optimizer='adam')
#     model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=5,batch_size=64,verbose=1)

#     last_date = pd.to_datetime("2023-2-15")
#     lst_output_1=[]
#     lst_output_2=[]

#     diff_1 = Buying_date-last_date
#     diff_2 = Selling_date-last_date
#     if int(diff_1.days)>0:
#         n_steps=100
#         x_input=test_data[len(test_data)-n_steps:].reshape(1,-1)
#         temp_input=list(x_input)
#         temp_input=temp_input[0].tolist()
#         i=0
#         while(i<int(diff_1.days)):
#             if(len(temp_input)>100):
#                 #print(temp_input)
#                 x_input=np.array(temp_input[1:])
#     #                 print("{} day input {}".format(i,x_input))
#                 x_input=x_input.reshape(1,-1)
#                 x_input = x_input.reshape((1, n_steps, 1))
#                 #print(x_input)
#                 yhat = model.predict(x_input, verbose=0)
#     #                 print("{} day output {}".format(i,yhat))
#                 temp_input.extend(yhat[0].tolist())
#                 temp_input=temp_input[1:]
#                 #print(temp_input)
#                 lst_output_1.extend(yhat.tolist())
#                 i=i+1
#             else:
#                 x_input = x_input.reshape((1, n_steps,1))
#                 yhat = model.predict(x_input, verbose=0)
#     #                 print(yhat[0])
#                 temp_input.extend(yhat[0].tolist())
#     #                 print(len(temp_input))
#                 lst_output_1.extend(yhat.tolist())
#                 i=i+1


            
#     if int(diff_2.days)>0:
#         n_steps=100
#         x_input=test_data[len(test_data)-n_steps:].reshape(1,-1)
#         temp_input=list(x_input)
#         temp_input=temp_input[0].tolist()
#         i=0
#         while(i<int(diff_2.days)):
#             if(len(temp_input)>100):
#                 #print(temp_input)
#                 x_input=np.array(temp_input[1:])
#     #                 print("{} day input {}".format(i,x_input))
#                 x_input=x_input.reshape(1,-1)
#                 x_input = x_input.reshape((1, n_steps, 1))
#                 #print(x_input)
#                 yhat = model.predict(x_input, verbose=0)
#     #                 print("{} day output {}".format(i,yhat))
#                 temp_input.extend(yhat[0].tolist())
#                 temp_input=temp_input[1:]
#                 #print(temp_input)
#                 lst_output_2.extend(yhat.tolist())
#                 i=i+1
#             else:
#                 x_input = x_input.reshape((1, n_steps,1))
#                 yhat = model.predict(x_input, verbose=0)
#     #                 print(yhat[0])
#                 temp_input.extend(yhat[0].tolist())
#     #                 print(len(temp_input))
#                 lst_output_2.extend(yhat.tolist())
#                 i=i+1
#     if int(diff_2.days)>0 and int(diff_1.days)>0:     
#       invests = scaler.inverse_transform(lst_output_1)[-1]
#       returns = scaler.inverse_transform(lst_output_2)[-1]
#       Final_Amount = ((returns-invests)*(int(Number_of_share)))
#     elif int(diff_2.days)>0 and int(diff_1.days)<0: 
#       invests = (df[(df['Symbol'] == Company_name) & (df['Date'] == Buying_date)]['Close'].values[0]).reshape(1,-1)
#       invests= invests[-1] * (int(Number_of_share))
#       returns = (scaler.inverse_transform(lst_output_2)[-1])* (int(Number_of_share))
#       Final_Amount = (returns-invests)
#     else:
#       invests = (df[(df['Symbol'] == Company_name) & (df['Date'] == Buying_date)]['Close'].values[0]).reshape(1,-1)
#       invests= invests[-1] * (int(Number_of_share))
#       returns = (df[(df['Symbol'] == Company_name) & (df['Date'] == Selling_date)]['Close'].values[0]).reshape(1,-1)
#       returns= returns[-1] * (int(Number_of_share))
      
#       Final_Amount = (returns-invests)


#     # In[2]:

#     if Final_Amount[0]>0:
#         print("Profit: " ,Final_Amount[0])
#         N50.update('B9', Final_Amount[0])
#         return Final_Amount[0]
#     else:
#         print("Loss: " ,Final_Amount[0])
#         N50.update('B9', Final_Amount[0])
#         return Final_Amount[0]

In [11]:
def stockPredictionModel(Company_name, Number_of_share, Buying_date, Selling_date):
    df = pd.read_csv('Nifty50.csv')
    df.head()
    df["Date"] = pd.to_datetime(df["Date"])
    # name_of_stock = input("Enter Name Of Stock")
    # number_of_share = int(input("Number Of Share"))
    Y = df[df["Symbol"]==Company_name][["Close"]]
    # start_date = input("Start Date")
    # end_date = input("End Date")
    Buying_date = pd.to_datetime(Buying_date)
    Selling_date = pd.to_datetime(Selling_date)



    scaler=MinMaxScaler(feature_range=(0,1))
    Y=scaler.fit_transform(np.array(Y).reshape(-1,1))



    training_size=int(len(Y)*0.65)
    test_size=len(Y)-training_size
    train_data,test_data=Y[0:training_size,:],Y[training_size:len(Y),:1]


    # convert an array of values into a dataset matrix
    def create_dataset(dataset, time_step):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_step-1):
            a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
            dataX.append(a)
            dataY.append(dataset[i + time_step, 0])
        return np.array(dataX), np.array(dataY)


    time_step = 100
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, ytest = create_dataset(test_data, time_step)


    X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
    X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

    ### Create the Stacked LSTM model
    model=Sequential()
    model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
    model.add(LSTM(50,return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='adam')
    model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=5,batch_size=64,verbose=1)

    last_date = pd.to_datetime("2023-2-15")
    final_output_1=[]
    final_output_2=[]

    Buy_date = Buying_date-last_date
    Sell_date = Selling_date-last_date
    if int(Buy_date.days)>0:
        n_steps=100
        x_input=test_data[len(test_data)-n_steps:].reshape(1,-1)
        temp_input=list(x_input)
        temp_input=temp_input[0].tolist()
        i=0
        while(i<int(Buy_date.days)):
            if(len(temp_input)>100):
                #print(temp_input)
                x_input=np.array(temp_input[1:])
    #                 print("{} day input {}".format(i,x_input))
                x_input=x_input.reshape(1,-1)
                x_input = x_input.reshape((1, n_steps, 1))
                #print(x_input)
                yhat = model.predict(x_input, verbose=0)
    #                 print("{} day output {}".format(i,yhat))
                temp_input.extend(yhat[0].tolist())
                temp_input=temp_input[1:]
                #print(temp_input)
                final_output_1.extend(yhat.tolist())
                i=i+1
            else:
                x_input = x_input.reshape((1, n_steps,1))
                yhat = model.predict(x_input, verbose=0)
    #                 print(yhat[0])
                temp_input.extend(yhat[0].tolist())
    #                 print(len(temp_input))
                final_output_1.extend(yhat.tolist())
                i=i+1


            
    if int(Sell_date.days)>0:
        n_steps=100
        x_input=test_data[len(test_data)-n_steps:].reshape(1,-1)
        temp_input=list(x_input)
        temp_input=temp_input[0].tolist()
        i=0
        while(i<int(Sell_date.days)):
            if(len(temp_input)>100):
                #print(temp_input)
                x_input=np.array(temp_input[1:])
    #                 print("{} day input {}".format(i,x_input))
                x_input=x_input.reshape(1,-1)
                x_input = x_input.reshape((1, n_steps, 1))
                #print(x_input)
                yhat = model.predict(x_input, verbose=0)
    #                 print("{} day output {}".format(i,yhat))
                temp_input.extend(yhat[0].tolist())
                temp_input=temp_input[1:]
                #print(temp_input)
                final_output_2.extend(yhat.tolist())
                i=i+1
            else:
                x_input = x_input.reshape((1, n_steps,1))
                yhat = model.predict(x_input, verbose=0)
    #                 print(yhat[0])
                temp_input.extend(yhat[0].tolist())
    #                 print(len(temp_input))
                final_output_2.extend(yhat.tolist())
                i=i+1
    if int(Sell_date.days)>0 and int(Buy_date.days)>0:     
      invests = scaler.inverse_transform(final_output_1)[-1]
      returns = scaler.inverse_transform(final_output_2)[-1]
      Final_Amount = ((returns-invests)*(int(Number_of_share)))
    elif int(Sell_date.days)>0 and int(Buy_date.days)<0: 
      invests = (df[(df['Symbol'] == Company_name) & (df['Date'] == Buying_date)]['Close'].values[0]).reshape(1,-1)
      invests= invests[-1] * (int(Number_of_share))
      returns = (scaler.inverse_transform(final_output_2)[-1])* (int(Number_of_share))
      Final_Amount = (returns-invests)
    else:
      invests = (df[(df['Symbol'] == Company_name) & (df['Date'] == Buying_date)]['Close'].values[0]).reshape(1,-1)
      invests= invests[-1] * (int(Number_of_share))
      returns = (df[(df['Symbol'] == Company_name) & (df['Date'] == Selling_date)]['Close'].values[0]).reshape(1,-1)
      returns= returns[-1] * (int(Number_of_share))
      
      Final_Amount = (returns-invests)


    # In[2]:

    if Final_Amount[0]>0:
        print("Profit: " ,Final_Amount[0])
        N50.update('B9', Final_Amount[0])
        return Final_Amount[0]
    else:
        print("Loss: " ,Final_Amount[0])
        N50.update('B9', Final_Amount[0])
        return Final_Amount[0]

In [12]:
stockPredictionModel('ADANIENT',30,'2023-10-1', '2023-12-12')

Epoch 1/5
9/9 [==============================] - 18s 764ms/step - loss: 0.0044 - val_loss: 0.0135
Epoch 2/5
9/9 [==============================] - 3s 393ms/step - loss: 8.2684e-04 - val_loss: 0.0083
Epoch 3/5
9/9 [==============================] - 3s 372ms/step - loss: 4.5224e-04 - val_loss: 0.0084
Epoch 4/5
9/9 [==============================] - 3s 370ms/step - loss: 3.3005e-04 - val_loss: 0.0084
Epoch 5/5
9/9 [==============================] - 3s 374ms/step - loss: 2.4895e-04 - val_loss: 0.0131
Loss:  -695.9406667946564


-695.9406667946564

In [14]:
stockPredictionModel('WIPRO', 30,'2020-2-25', '2022-11-30')

Epoch 1/5
9/9 [==============================] - 17s 658ms/step - loss: 0.0456 - val_loss: 0.0107
Epoch 2/5
9/9 [==============================] - 3s 338ms/step - loss: 0.0093 - val_loss: 0.0039
Epoch 3/5
9/9 [==============================] - 3s 336ms/step - loss: 0.0041 - val_loss: 0.0029
Epoch 4/5
9/9 [==============================] - 3s 342ms/step - loss: 0.0020 - val_loss: 0.0114
Epoch 5/5
9/9 [==============================] - 3s 342ms/step - loss: 0.0015 - val_loss: 0.0035
Profit:  4883.999639999999


4883.999639999999

In [15]:
stockPredictionModel('WIPRO', 30,'2023-1-31', '2025-12-1')

Epoch 1/5
9/9 [==============================] - 19s 736ms/step - loss: 0.0659 - val_loss: 0.0430
Epoch 2/5
9/9 [==============================] - 3s 327ms/step - loss: 0.0129 - val_loss: 0.0113
Epoch 3/5
9/9 [==============================] - 3s 328ms/step - loss: 0.0048 - val_loss: 0.0096
Epoch 4/5
9/9 [==============================] - 3s 322ms/step - loss: 0.0020 - val_loss: 0.0103
Epoch 5/5
9/9 [==============================] - 3s 330ms/step - loss: 0.0016 - val_loss: 0.0043
Loss:  -6988.716428521634


-6988.716428521634

In [13]:
while(True):
    Company_name = N50.acell('B4').value
    Number_of_share = N50.acell('B5').value
    Buying_date = N50.acell('B6').value
    Selling_date = N50.acell('B7').value

    stockPredictionModel(Company_name, Number_of_share, Buying_date, Selling_date)
    time.sleep(5)


Epoch 1/5
9/9 [==============================] - 19s 799ms/step - loss: 0.0064 - val_loss: 0.0338
Epoch 2/5
9/9 [==============================] - 3s 364ms/step - loss: 0.0012 - val_loss: 0.0135
Epoch 3/5
9/9 [==============================] - 3s 378ms/step - loss: 4.2034e-04 - val_loss: 0.0113
Epoch 4/5
9/9 [==============================] - 3s 358ms/step - loss: 2.8396e-04 - val_loss: 0.0087
Epoch 5/5
9/9 [==============================] - 3s 352ms/step - loss: 2.5281e-04 - val_loss: 0.0128
Loss:  -1665.9322897009133
Epoch 1/5


Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x0000020AB45A8940>
Traceback (most recent call last):
  File "C:\Users\ashut\Downloads\anaconda\lib\weakref.py", line 371, in remove
    self = selfref()
KeyboardInterrupt: 


9/9 [==============================] - 19s 703ms/step - loss: 0.0090 - val_loss: 0.0225
Epoch 2/5
9/9 [==============================] - 3s 340ms/step - loss: 0.0013 - val_loss: 0.0452
Epoch 3/5
9/9 [==============================] - ETA: 0s - loss: 5.4704e-04

KeyboardInterrupt: 